# variables

In [2]:
API_URL = "https://api.osparc"
WEB_API_URL = "https://osparc"
USERNAME = "THEUSER"
PASSWORD = "THEPASSWORD"
API_KEY = "b02ee1b0-b089-59f0-bd14-44919b2fa165"
API_SECRET = "2cf84558-d757-5a1b-a351-1c81b5f2d93d"
SOLVER_NAME = "simcore/services/comp/itis/sleeper"
SOLVER_VERSION = "2.1.6"

CLUSTER_ID = 0 # run on the default cluster


# initialize connection

In [3]:
from multiprocessing.pool import AsyncResult
from tenacity import AsyncRetrying, TryAgain, retry_if_exception_type
from tenacity.wait import wait_fixed
from tqdm.notebook import tqdm
import asyncio
import functools
import json
import osparc
import typing
import urllib3


cfg = osparc.Configuration(
    host=f"{API_URL}",
    username=API_KEY,
    password=API_SECRET,
)
# cfg.debug = True

In [4]:
with osparc.ApiClient(cfg) as api_client:
    profile = osparc.UsersApi(api_client).get_my_profile()
    solvers_api = osparc.SolversApi(api_client)
    original_api_client_call_api = solvers_api.api_client.call_api

    def patched_api_client_for_starting_jobs(*args, **kwargs):
        # print("called with: ", args, kwargs)
        if args[0] == "/v0/solvers/{solver_key}/releases/{version}/jobs/{job_id}:start" and args[1] == "POST":
            query_params = args[3]
            # print("called to start a job, let's add ", CLUSTER_ID, "to", query_params)
            query_params.append(("cluster_id", CLUSTER_ID))
            # print("changed to: ", args, kwargs)
        return original_api_client_call_api(*args, **kwargs)

    solvers_api.api_client.call_api = patched_api_client_for_starting_jobs

# get solver

In [ ]:
solver = typing.cast(osparc.Solver, solvers_api.get_solver_release(SOLVER_NAME, SOLVER_VERSION))
print(solver)

# create job

In [6]:
async def create_job(solver: osparc.Solver) -> osparc.Job:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.create_job,
            solver.id,
            solver.version,
            osparc.JobInputs(
                {
                    "input_2": 15,
                    "input_4": 0
                }
            ), async_req=True
        ))
    assert isinstance(result, AsyncResult) # nosec
    # print(job)
    return typing.cast(osparc.Job, await asyncio.get_running_loop().run_in_executor(None, result.get))

# list jobs

In [ ]:
async def list_jobs(solver: osparc.Solver) -> list[osparc.Job]:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.list_jobs, solver.id, solver.version, async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    # print([job.id for job in jobs])
    return typing.cast(list[osparc.Job], await asyncio.get_running_loop().run_in_executor(None, result.get))
jobs =await list_jobs(solver)
assert len(jobs) == 0, f"found {len(jobs)} jobs"

# inspect job

In [18]:
async def inspect_job(solver: osparc.Solver, job: osparc.Job)-> osparc.JobStatus:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.inspect_job, solver.id, solver.version, job.id,async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    # print(status)
    return typing.cast(osparc.JobStatus, await asyncio.get_running_loop().run_in_executor(None, result.get))

## get job result

In [19]:
async def get_job_result(solver: osparc.Solver, job: osparc.Job) -> osparc.JobOutputs:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.get_job_outputs, solver.id, solver.version, job.id, async_req=True))
    assert isinstance(result, AsyncResult) # nosecregistry.staging.osparc.io/simcore/services/comp/itis/sleeper:2.0.2
    return typing.cast(osparc.JobOutputs, await asyncio.get_running_loop().run_in_executor(None, result.get))

# start job

In [20]:
async def start_job(solver: osparc.Solver, job: osparc.Job) -> osparc.JobStatus:
    result=  await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.start_job, solver.id, solver.version, job.id,async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    return typing.cast(osparc.JobStatus, await asyncio.get_running_loop().run_in_executor(None, result.get))

# delete a job corresponding project

In [21]:
def _login_webserver() -> dict[str, str]:
   # create a PoolManager instance
    http = urllib3.PoolManager()

    # define the URL of the login page
    url = f'{WEB_API_URL}/v0/auth/login'

    # define the data to be sent with the POST request
    data = {'email': USERNAME, 'password': PASSWORD}

    # send the POST request to the login page
    response = http.request('POST', url, body=json.dumps(data), headers={'Content-Type': 'application/json'})

    # print the response status code and content
    if response.status != 200:
        raise RuntimeError("failed to login!")
    
    # get the cookies
    cookies = response.headers.get("Set-Cookie")
    return cookies

In [22]:
def delete_jobs(jobs: list[osparc.Job]) -> None:
    # create a PoolManager instance
    http = urllib3.PoolManager()

    # define the URL of the login page
    url = f'{WEB_API_URL}/v0/auth/login'

    # define the data to be sent with the POST request
    data = {'email': USERNAME, 'password': PASSWORD}

    # send the POST request to the login page
    response = http.request('POST', url, body=json.dumps( data), headers={'Content-Type': 'application/json'})

    # print the response status code and content
    if response.status != 200:
        raise RuntimeError("failed to login!")
    # print("logged in, proceeding with deletion...")

    # get the cookies
    cookies = response.headers.get("Set-Cookie")

    # delete a project
    for job in tqdm(jobs, desc="Deleting jobs"):
        url = f'{WEB_API_URL}/v0/projects/{job.id}'
        response = http.request("DELETE", url, headers={'Content-Type': 'application/json', "Cookie": cookies})
        # print(f"job {job.id} is {'deleted' if response.status == 204 else 'failed to delete'}")


### list jobs through webserver

In [23]:
def list_job_ids_from_web_server() -> list[str]:
    cookies = _login_webserver()
    # create a PoolManager instance
    http = urllib3.PoolManager()

    # list projects that are hidden
    url = f'{WEB_API_URL}/v0/projects?type=user&show_hidden=true'
    offset = 0
    limit = 30
    response = http.request("GET", f"{url}&limit={limit}&offset={offset}", headers={'Content-Type': 'application/json', "Cookie": cookies})
    if response.status != 200:
        raise RuntimeError(f"Unable to list projects from webserver: {response.data}")
    data = json.loads(response.data)

    total = data["_meta"]["total"]
    total_count = data["_meta"]["count"]
    offset += data["_meta"]["count"]
    list_of_job_ids = [project["uuid"] for project in data["data"] if str(project["name"]).startswith("solvers/simcore")]

    while total_count < total:
        response = http.request("GET", f"{url}&limit={limit}&offset={offset}", headers={'Content-Type': 'application/json', "Cookie": cookies})
        if response.status != 200:
            raise RuntimeError(f"Unable to list projects from webserver: {response.data}")
        data = json.loads(response.data)
        list_of_job_ids += [project["uuid"] for project in data["data"] if str(project["name"]).startswith("solvers/simcore")]
        total_count += data["_meta"]["count"]
        offset+=data["_meta"]["count"]
    
    return list_of_job_ids

list_of_job_ids = list_job_ids_from_web_server()
print(len(list_of_job_ids))

20


## Fake jobs to delete them

In [13]:
jobs = [osparc.Job(id=job_id, name="fake", inputs_checksum="", created_at="", runner_name="blah", url="asd", runner_url="asd", outputs_url="asd") for job_id in list_of_job_ids]

# run N sleepers

In [ ]:
NUM_JOBS = 20

jobs = []
with tqdm(total=NUM_JOBS, desc="Creating jobs") as pbar:
    for result in asyncio.as_completed([create_job(solver) for _ in range(NUM_JOBS)]):
        job = await result
        jobs.append(job)
        pbar.update()

In [ ]:
async def run_job(solver: osparc.Solver, job: osparc.Job):
    with tqdm(total=100) as pbar:
        await start_job(solver, job)

        job_status = await inspect_job(solver, job)
        assert job_status.progress is not None # nosec
        pbar.update(job_status.progress)
        current_progress = job_status.progress
        async for attempt in AsyncRetrying(wait=wait_fixed(1), retry=retry_if_exception_type()):
            with attempt:
                job_status = await inspect_job(solver, job)
                assert job_status.progress is not None # nosec
                if job_status.progress != current_progress:
                    pbar.update(job_status.progress - current_progress)
                    current_progress = job_status.progress
                    
                if not (job_status.progress == 100 and job_status.state in ["FAILED", "SUCCESS"]):
                    raise TryAgain
                
        if job_status.state == "FAILED":
            raise RuntimeError("job failed!")
        return job_status

job_statuses = []
with tqdm(total=len(jobs), desc="Running jobs") as pbar:
    for result in asyncio.as_completed([run_job(solver, job) for job in jobs]):
        try:
            job_statuses.append(await result)            
        except osparc.ApiException as exc:
            tqdm.write(f"Error while running {job.id}: {exc}")
        finally:
            pbar.update()

## Check results

In [39]:
assert all(status.state == "SUCCESS" for status in job_statuses)

# Get job submission timestamps

In [ ]:
print(job_statuses[100])

## Get job result

In [ ]:
await get_job_result(solver, jobs[0])

In [ ]:
# jobs =await list_jobs(solver)
delete_jobs(jobs)
await asyncio.sleep(2)
jobs =await list_jobs(solver)
assert len(jobs) == 0, f"found {len(jobs)} jobs"